In [ ]:
import requests
import torch
import bitsandbytes as bnb
import transformers
import sentencepiece
import accelerate
import openai
import httpx

In [ ]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [ ]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
audio_filename="/content/drive/MyDrive/denver_extract.mp3"

In [ ]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import time

In [ ]:
def transcribe_audio(audio_file):
    try:
        with open(audio_file, "rb") as f:
            transcript_response = openai.audio.transcriptions.create(model="whisper-1", file=f)
        return transcript_response.text
    except Exception as e:
        return f"❌ Error in transcription: {str(e)}"

# ------------------------------
# Streaming generator function
def stream_minutes(transcript):
    try:
        messages = [
            {
                "role": "system",
                "content": "You are an assistant that produces detailed meeting minutes in markdown format including summary, attendees, key discussion points, takeaways, and action items with owners."
            },
            {
                "role": "user",
                "content": f"Below is a transcript of a meeting. Please write the minutes in markdown format:\n\n{transcript}"
            }
        ]

        inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

        generated_ids = model.generate(
            inputs,
            max_new_tokens=2000,
            do_sample=False,
            return_dict_in_generate=True,
            output_scores=False
        ).sequences[0]

        output = ""
        for i in range(inputs.shape[1], len(generated_ids)):
            token_id = generated_ids[i].item()
            output += tokenizer.decode([token_id], skip_special_tokens=True)
            yield output
            time.sleep(0.02)

    except Exception as e:
        yield f"❌ Error while generating minutes: {str(e)}"

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
!pip install gradio
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## 🎙️ Meeting Minutes Generator with Streaming")

    audio_input = gr.Audio(label="Upload Meeting Audio", type="filepath")
    transcript_box = gr.Textbox(label="Transcript", lines=10)
    minutes_markdown = gr.Markdown(label="📋 Minutes (Markdown Streaming)")

    transcribe_btn = gr.Button("Transcribe")
    stream_btn = gr.Button("Generate Meeting Minutes")

    transcribe_btn.click(fn=transcribe_audio, inputs=audio_input, outputs=transcript_box)
    stream_btn.click(fn=stream_minutes, inputs=transcript_box, outputs=minutes_markdown)

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://af24aa57a88b783fbd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
